# Data Simulation - Part I: Feature Space

In [2]:
##### Simulate Data for Reserve Calculation #####

# Option practical: Age of contract is simulated as a random share of the respective duration
# Option all-over: All contracts are simulated as random values on their Min-Max Range

def data_simulation_features(A, B, c, N_contracts, Max_min, N_features=4, option = 'practical'):
   
    data = np.zeros([N_contracts,N_features])


    # Use Sobol Sequence to optimize uniform coverage of multidimensional feature space
    random.seed(42)
    if option == 'practical':
        data[:,0:-1] = sobol_seq.i4_sobol_generate(N_features-1,N_contracts) # simulate age_of_contract seperately, conditionally on the duration
    elif option == 'all-over': # allows for irreasonable, i.e. matured contracts
        data = sobol_seq.i4_sobol_generate(N_features,N_contracts)
    else:
        print('Option unknown!')
        pass
    
    # Simulate initial ages of policyholders upon signing the contract
    # Note: We first simulate the initial age at start of contract (-> use age_up = Max_min[0,1]-Max_min[2,0])
    data[:,0] =(Max_min[0,0] + (Max_min[0,1]-Max_min[2,1]-Max_min[0,0])*data[:,0])

    # simulate sums insured
    data[:,1] = (Max_min[1,0]+(Max_min[1,1]-Max_min[1,0])*data[:,1])

    # simulate duration
    data[:,2] = (Max_min[2,0]+(Max_min[2,1]-Max_min[2,0])*data[:,2])
    
    if option == 'practical':
        # simulate age of contract in relative terms
        rnd = np.random.rand(N_contracts)
        # random percentage of duration has passed; at least one remaining year of contract (hence dur - 1)
        data[:,3] = rnd*(data[:,2]-1)         
    elif option == 'all-over':
        data[:,3] = (Max_min[3,0]+(Max_min[3,1]-Max_min[3,0])*data[:,3])
    else:
        print('Option unknown!')
        pass
    
    # obtain current age of policyholder (initial age + age of contract)
    data[:,0] = data[:,0] + data[:,3]
    
    # ceil data as integers
    data = data.astype('int')
    
    
    ### Lower-dimensional Datasets
    data_age = data[:,0]
    data_sum = data[:,1]
    data_dur = data[:,2]
    data_aoc = data[:,3];
    
    return [data, [data_age, data_sum, data_dur, data_aoc]]